In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

import pycollocation

import inputs
import models
import shooting

Make sure working with version `0.2.3a0` of `pycollocation`...

In [13]:
pycollocation.__version__

'0.2.3a0'

## Worker skill and firm productivity are $\sim U[a, b]$...

In [3]:
# define some workers skill
x, a, b = sym.var('x, a, b')
skill_cdf = (x - a) / (b - a)
skill_params = {'a': 1.0, 'b': 2.0}
skill_bounds = [skill_params['a'], skill_params['b']]

workers = inputs.Input(var=x,
                       cdf=skill_cdf,
                       params=skill_params,
                       bounds=skill_bounds,
                       )

# define some firms
y = sym.var('y')
productivity_cdf = (y - a) / (b - a)
productivity_params = skill_params
productivity_bounds = skill_bounds

firms = inputs.Input(var=y,
                     cdf=productivity_cdf,
                     params=productivity_params,
                     bounds=productivity_bounds,
                     )

## ...and production function is multiplicatively separable.
A particularly attractive case arises under multiplicative separability of the form

$$F(x, y, l, r) = A(x, y)B(l, r)$$

In this case the condition for positive assortative matching can be written as 

$$\frac{AA_{xy}}{A_xA_y}\frac{BB_{lr}}{B_lB_r} \ge 1$$

If $B$ has constant elasticity of substitution form, then we obtain an even simpler condition

$$\frac{AA_{xy}}{A_xA_y} \frac{1}{\sigma_{lr}}\ge 1$$

where $\sigma_{lr}$ is the elasticity of substitution between $l$ and $r$. 

Finally, if $A$ also has constant elasticity of substitution form, then we obtain an even simpler condition

$$\frac{1}{\sigma_{xy}\sigma_{lr}}\ge 1 \iff \sigma_{xy}\sigma_{lr} \le 1$$

where $\sigma_{xy}$ is the elasticity of substitution between $x$ and $y$.

In [4]:
# define symbolic expression for CES between x and y
omega_A, sigma_A = sym.var('omega_A, sigma_A')
A = ((omega_A * x**((sigma_A - 1) / sigma_A) + 
     (1 - omega_A) * y**((sigma_A - 1) / sigma_A))**(sigma_A / (sigma_A - 1))) 

# define symbolic expression for Cobb-Douglas between l and r
l, r, omega_B, sigma_B = sym.var('l, r, omega_B, sigma_B')
B = l**omega_B * r**(1 - omega_B)

F = A * B

# Example:

More sophisticated production function that is not multiplicatively separable...

In [106]:
# define some workers skill
x, mu1, sigma1 = sym.var('x, mu1, sigma1')
skill_cdf = 0.5 + 0.5 * sym.erf((sym.log(x) - mu1) / sym.sqrt(2 * sigma1**2))
skill_params = {'mu1': 0.0, 'sigma1': 1e0}
skill_bounds = [1e-3, 50.0]

workers = inputs.Input(var=x,
                       cdf=skill_cdf,
                       params=skill_params,
                       bounds=skill_bounds,
                       )

# define some firms
y, mu2, sigma2 = sym.var('y, mu2, sigma2')
productivity_cdf = 0.5 + 0.5 * sym.erf((sym.log(y) - mu2) / sym.sqrt(2 * sigma2**2))
productivity_params = {'mu2': 0.0, 'sigma2': 1e0}
productivity_bounds = [1e-3, 50.0]

firms = inputs.Input(var=y,
                     cdf=productivity_cdf,
                     params=productivity_params,
                     bounds=productivity_bounds,
                     )

# define some valid model params
F_params = {'eta': 0.89, 'kappa': 1.0, 'gamma': 0.54, 'rho': 0.24, 'A': 1.0, 'k': 1.0}

# define a valid production function
A, k, kappa, eta, rho, l, gamma, r = sym.var('A, k, kappa, eta, rho, l, gamma, r')
F = r * A * kappa * eta * ((k * x)**rho + (1 - eta) * (y * (l / r))**rho)**(gamma / rho)

model = models.Model('negative',
                     workers=workers,
                     firms=firms,
                     production=F,
                     params=F_params)

In [108]:
solver = solvers.ShootingSolver(model=model)

In [109]:
solver.solve(1e0, tol=1e-4, number_knots=10000, integrator='vode',
             atol=1e-9, rtol=1e-6, check=True)

Exhausted firms: initial guess of 0.5 for firm size is too low.


In [110]:
solver.integrator.successful()

False

In [111]:
solver._solution

array([[  1.00000000e-03,   5.00000000e+01,   7.50000000e-01,
          6.25292214e-02,   1.02985470e-01],
       [  6.00040004e-03,   4.91355141e+01,   2.93698995e-02,
          6.50895813e-01,   1.02614135e-01],
       [  1.10008001e-02,   1.87728995e+01,   6.10881218e-04,
          8.73207163e+00,   9.34543423e-02],
       [  1.34286568e-02,   2.61227847e+06,   1.98653860e+04,
          1.04151668e-01,   1.79568793e+03]])

In [46]:
integrat

50.0